> Run this notebook at the root directory

In [19]:
import os, sys, glob
import random
import torch, numpy as np
import trimesh
import time
from PIL import Image,ImageFont, ImageDraw
from random import choice,choices
sys.path.append("..")
from lib.dataset import ProjectOnSphere, make_sgrid,rnd_rot,rotmat

In [20]:
def find_classes_in_dataset(root="/home/qiangzibro/caps3d/data/modelnet40/modelnet40_train/"):
    files = sorted(glob.glob(os.path.join(root, '*.off')))
    labels = {}
    c = set()
    for fpath in files:
        fname = os.path.splitext(os.path.basename(fpath))[0]
        c_ = "_".join(fname.split('_')[:-1])  # extract label.
        c.add(c_)
        labels[fname] = c_
    return list(c)

def find_classes(name="modelnet40", train=True):
    if train:
        return find_classes_in_dataset(root=f"/home/qiangzibro/caps3d/data/{name}/{name}_train/")
    else:
        return find_classes_in_dataset(root=f"/home/qiangzibro/caps3d/data/{name}/{name}_test/")

In [21]:
datasets = ["modelnet10", "modelnet40", "shrec15_0.5", "shrec17"]
print("Dataset : # of dataset")
for d in datasets:
    print(d, ":", len(find_classes(d)))

Dataset : # of dataset
modelnet10 : 10
modelnet40 : 40
shrec15_0.5 : 48
shrec17 : 20


## 三维目标与求结构可视化

baseline中使用了下面这种方式处理面片

In [22]:
# class ToMesh:
#     def __init__(self, random_rotations=False, random_translation=0):
#         self.rot = random_rotations
#         self.tr = random_translation

#     def __call__(self, path, to_center=False):
#         mesh = trimesh.load_mesh(path)
#         if to_center:
#             mesh.apply_translation(-mesh.centroid)  
#         if self.tr > 0:
#             tr = np.random.rand() * self.tr
#             rot = rnd_rot()
#             mesh.apply_transform(rot)
#             mesh.apply_translation([tr, 0, 0])

#             if not self.rot:
#                 mesh.apply_transform(rot.T)

#         if self.rot:
#             mesh.apply_transform(rnd_rot())

#         r = np.max(np.linalg.norm(mesh.vertices, axis=-1))
#         mesh.apply_scale(0.99 / r)
#         return mesh

#     def __repr__(self):
#         return self.__class__.__name__ + '(rotation={0}, translation={1})'.format(self.rot, self.tr)

class ToMesh:
    def __init__(self, random_rotations=False, random_translation=0):
        self.rot = random_rotations
        self.tr = random_translation

    def __call__(self, path):
        mesh = trimesh.load_mesh(path)
        mesh.remove_degenerate_faces()
        mesh.fix_normals()
        mesh.fill_holes()
        mesh.remove_duplicate_faces()
        mesh.remove_infinite_values()
        mesh.remove_unreferenced_vertices()

        mesh.apply_translation(-mesh.centroid)

        r = np.max(np.linalg.norm(mesh.vertices, axis=-1))
        mesh.apply_scale(1 / r)

        if self.tr > 0:
            tr = np.random.rand() * self.tr
            rot = rnd_rot()
            mesh.apply_transform(rot)
            mesh.apply_translation([tr, 0, 0])

            if not self.rot:
                mesh.apply_transform(rot.T)

        if self.rot:
            mesh.apply_transform(rnd_rot())

        r = np.max(np.linalg.norm(mesh.vertices, axis=-1))
        mesh.apply_scale(0.99 / r)

        return mesh

    def __repr__(self):
        return self.__class__.__name__ + '(rotation={0}, translation={1})'.format(self.rot, self.tr)


In [23]:
# 先来看看原本模型长啥样
path = "../data/modelnet40/modelnet40_train/piano_0006.off"
mesh = trimesh.load_mesh(path)
mesh.show()

In [24]:
# 经过论文中转换的方式
mesh = ToMesh(False)(path)
mesh.show()

In [27]:
# 把模型放在一个单位球里
sgrid = make_sgrid(b=32, alpha=0, beta=0, gamma=0)
scene = trimesh.Scene([
    mesh,
    trimesh.load_path(sgrid)
])
scene.show()

In [29]:
from lib.dataset.transforms import ToMesh, ProjectOnSphere
from mayavi import mlab
mlab.figure(1, bgcolor=(1, 1, 1), fgcolor=(0, 0, 0), size=(1000, 800))

# Make sphere, choose colors
phi, theta = np.mgrid[0: np.pi: 64j, 0: 2 * np.pi: 64j]
x, y, z = np.sin(phi) * np.cos(theta), np.sin(phi) * np.sin(theta), np.cos(phi)
f = ProjectOnSphere(32, "shrec15_0.5", normalize=False)(mesh)
f = f[0, :, :]
mlab.mesh(x, y, z, scalars=f, colormap="gray")
mlab.view()
mlab.show()

## 球卷积的可视化
- `with_preprocess(path, b=32, k=None)`
函数能看到预处理后，模型位于求里面的样子
并且可以发射若干条射线，检查与模型的交点

- `without_preprocess(path, b=32, k=None)`
直接对原始模型进行可视化

In [10]:
def set_color(mesh, index_tri):
    # unmerge so viewer doesn't smooth
    mesh.unmerge_vertices()
    # make mesh white- ish
    mesh.visual.face_colors = [255,255,255,255]
    mesh.visual.face_colors[index_tri] = [0, 255, 0, 255]


def make_insects(mesh, sgrid, index, b=32):
    # 光线投射
    ray_origins, ray_directions = sgrid[index], -sgrid[index]
    index_tri, index_ray, locations = mesh.ray.intersects_id(
            ray_origins=ray_origins, 
            ray_directions=ray_directions, 
            multiple_hits=False, 
            return_locations=True)

#     print('The rays with index: {} hit the triangles stored at mesh.faces[{}]'.format(index_ray, index_tri))
    print('总共有{}条光线打在了物体上(总共:{}条光线)'.format(len(index_ray), len(sgrid)))
    return ray_origins, ray_directions,index_tri, index_ray, locations

def make_scene_without_lines(mesh, sgrid):
    scene = trimesh.Scene([
        mesh,
        trimesh.load_path(sgrid),
    ])
    return scene
def make_scene(mesh, sgrid, index, b=32):
    ray_origins, ray_directions,index_tri, index_ray, locations=make_insects(mesh, sgrid, index, b)
       
    # 组合成一个场景 
    ray_visualize = trimesh.load_path(np.hstack((
        ray_origins,
        [[0,0,0] for i in range(len(index))]
    )).reshape(-1, 2, 3))
    set_color(mesh, index_tri)
    scene = trimesh.Scene([
        mesh,
        trimesh.load_path(sgrid),
        ray_visualize
    ])
    return scene


def with_preprocess(path, b=32, k=None):
    """使用论文中处理面片的方式"""
    sgrid = make_sgrid(b=b, alpha=0, beta=0, gamma=0)
    index = random.choices(range(len(sgrid)), k=len(sgrid) if k is None else k) #随机选取k个点进行可视化
    mesh = ToMesh(True, 0)(path)
    
    return make_scene(mesh, sgrid, index, b=b)

def without_preprocess(path, b=32, k=None):
    """不使用论文中处理面片的方式，直接加载"""
    sgrid = make_sgrid(b=b, alpha=0, beta=0, gamma=0)
    index = random.choices(range(len(sgrid)), k=len(sgrid) if k is None else k) #随机选取k个点进行可视化
    mesh = trimesh.load_mesh(path)
    return make_scene(mesh,sgrid, index, b=b)

In [ ]:
with_preprocess(path, k=20).show()

In [ ]:
# 不进行预处理， 模型就放不到单位球里
without_preprocess(path, k=20).show()

## 将三维目标平移到球心
这样做是必要的，可以看到，将模型移到中心后，绝大数光线都能打在模型上，因此可以提供更多的特征。
只需要一行代码就可以把三维目标移到中心，方法：
```python
mesh.apply_translation(-mesh.centroid)
```
我们分别可视化原模型、处理后的模型、以及把模型移到中心点

In [ ]:
path="data/modelnet40/modelnet40_train/tv_stand_0001.off"
mesh = ToMesh(True, 0)(path)
# mesh = trimesh.load_mesh(path)
mesh.show()

In [ ]:
sgrid = make_sgrid(b=32, alpha=0, beta=0, gamma=0)

mesh = ToMesh(True, 0)(path) #trimesh.load_mesh(path)
print("平移前",mesh.center_mass)
scene = make_scene_without_lines(mesh, sgrid)
scene.show()

In [ ]:
print("平移后",mesh.center_mass)
mesh.apply_translation(-mesh.centroid)
scene = make_scene_without_lines(mesh, sgrid)
scene.show()

## 保存图片

In [51]:
def get_image_from_mesh(mesh, unit_sphere=False):
    if unit_sphere:
        sgrid = make_sgrid(b=32, alpha=0, beta=0, gamma=0)
        scene = make_scene_without_lines(mesh, sgrid)
    else:
        scene = trimesh.Scene([
            mesh
        ])
    data = scene.save_image(visible=True)
    rendered = Image.open(trimesh.util.wrap_as_stream(data)).convert('RGB')
    return rendered

def save_one_mesh(mesh=None, path=None, filename="test.jpg"):
    if not mesh and not path:
        raise Exception("wrong input! at least one input")
    if path:
        mesh = trimesh.load_mesh(path)
    get_image_from_mesh(mesh).save(filename)
         
def imgs_to_row(ims):
    """把一个PIL列表图片汇总成一行图片"""
    widths, heights = zip(*(i.size for i in ims))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in ims:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]
    return new_im

def imgs_to_col(ims):
    """把一个PIL列表图片汇总成一列图片"""
    widths, heights = zip(*(i.size for i in ims))

    total_width = max(widths)
    max_height = sum(heights)

    new_im = Image.new('RGB', (total_width, max_height))
    y_offset = 0
    for im in ims:
        new_im.paste(im, (0,y_offset))
        y_offset += im.size[1]
    return new_im

def save_images_in(dataset, k, train=True):
    paths=[
           choice(glob.glob(f"data/{dataset}/{dataset}_train/{cls}_*.off"))
          for cls in find_classes(dataset, train)
          ]
    random.shuffle(paths)
    
    paths = paths[:k]
    save_images(paths, dataset)

def draw_text_on(mesh, text, unit_sphere=False):
    img = get_image_from_mesh(mesh, unit_sphere)
    ImageDraw.Draw(img).text((500,100), text, fill = (255, 0 ,0), font=ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf",100))
    return img
def save_images(paths, filename="test.jpg"):
    IMAGES = []
    for mesh_object_path in paths:
        images = []
        name = mesh_object_path.split("/")[-1].split("_")[0]
        name = "logs/pics/"+name

        
        # 原模型
        mesh = trimesh.load_mesh(mesh_object_path)
        img = draw_text_on(mesh,name.split("/")[-1], False)
        images.append(img)

        # 我们使用的方式
        mesh = ToMesh(True, 0)(mesh_object_path, to_center=True)
        images.append(get_image_from_mesh(mesh, True))
        

        # ugscnn
        mesh = ToMesh1(True, 0)(mesh_object_path)
        images.append(get_image_from_mesh(mesh, True))
        
        IMAGES.append(images) 

    tmp = []
    for images in IMAGES:
        tmp.append(imgs_to_row(images))
    fig = imgs_to_col(tmp)
    fig.save(f"logs/pics/{filename}.jpg")
    
save_images(paths, "")

/usr/miniconda3/envs/capsules_torch12_py37/lib/python3.7/site-packages/trimesh/repair.py:258: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cycles = np.array(nx.cycle_basis(g))


每个数据集选取k个类型保存成图片

In [ ]:
# ! rm logs/pics/*
# import threading
# T = []
# datasets = ["modelnet10", "modelnet40", "shrec15", "shrec17"]
# k = 20
# for dataset in datasets:
#     save_images_in(dataset,k,False)
# #     t = threading.Thread(target=save_images_in, args=(dataset,k,False))
# #     t.start()
# #     T.append(t)
# # for t in T:
# #     t.join()
# #     save_images_in(dataset, k=k, train=False)
# ! tar czf logs/pics_"$k".tar.gz logs/pics

In [ ]:
# paths=["data/modelnet40/modelnet40_train/airplane_0001.off",
#        "data/modelnet40/modelnet40_train/person_0001.off",
#        "data/modelnet40/modelnet40_train/glass_box_0001.off"
#       ]

# def save_mesh_in_logs_pics(mesh_object_path):
#     name = mesh_object_path.split("/")[-1].split("_")[0]
#     name = "logs/pics/"+name

#     sgrid = make_sgrid(b=32, alpha=0, beta=0, gamma=0)
#     mesh = trimesh.load_mesh(mesh_object_path)
#     scene = trimesh.Scene([
#         mesh
#     ])
#     save_image(scene, name+"_original_model.jpg")

#     mesh = ToMesh(True, 0)(mesh_object_path)
#     scene = make_scene_without_lines(mesh, sgrid)
#     save_image(scene, name+"_not_to_origin.jpg")

#     mesh.apply_translation(-mesh.centroid)
#     scene = make_scene_without_lines(mesh, sgrid)
#     save_image(scene, name+"_to_origin.jpg")
    
# [save_mesh_in_logs_pics(path) for path in paths]
# ! tar czf logs/pics.tar.gz logs/pics

In [18]:
scores = {20: [0, 20, 0.0], 1: [4, 100, 0.04], 14: [0, 100, 0.0], 6: [49, 100, 0.49], 26: [1, 100, 0.01],
          39: [88, 100, 0.88], 10: [2, 20, 0.1], 17: [0, 100, 0.0], 5: [55, 86, 0.6395348837209303],
          18: [12, 86, 0.13953488372093023], 34: [89, 100, 0.89], 28: [0, 20, 0.0], 31: [27, 50, 0.54],
          24: [62, 100, 0.62], 21: [0, 20, 0.0], 0: [0, 100, 0.0], 8: [41, 100, 0.41], 16: [0, 100, 0.0],
          32: [82, 100, 0.82], 9: [32, 100, 0.32], 19: [24, 100, 0.24], 3: [0, 20, 0.0], 25: [0, 20, 0.0],
          38: [1, 100, 0.01], 35: [8, 100, 0.08], 4: [33, 100, 0.33], 23: [52, 86, 0.6046511627906976],
          7: [1, 100, 0.01], 22: [0, 20, 0.0], 36: [0, 20, 0.0], 27: [3, 20, 0.15], 37: [0, 20, 0.0],
          30: [0, 20, 0.0], 13: [0, 20, 0.0], 33: [0, 20, 0.0], 15: [0, 20, 0.0], 12: [0, 20, 0.0], 2: [0, 20, 0.0],
          11: [0, 20, 0.0], 29: [0, 20, 0.0]}

from collections import OrderedDict
from lib.dataset import  modelnet40_classes
scores=OrderedDict(sorted(scores.items()))
paths = []
dataset="modelnet40"

for k,v in scores.items():
    if v[2] == 0: #找出识别率为0的类别
        print(k, ":", modelnet40_classes[k])
        cls=modelnet40_classes[k]
        paths.append(choice(glob.glob(f"data/{dataset}/{dataset}_train/{cls}_*.off")))

0 : tv_stand
2 : lamp
3 : cup
11 : sink
12 : curtain
13 : wardrobe
14 : glass_box
15 : door
16 : range_hood
17 : mantel
20 : stairs
21 : bench
22 : bowl
25 : flower_pot
28 : xbox
29 : radio
30 : laptop
33 : person
36 : stool
37 : keyboard


In [44]:
trimesh.load_mesh(paths[-3]).show()

In [45]:
paths[-3]

'data/modelnet40/modelnet40_train/person_0016.off'

In [35]:
ImageFont.truetype?

Signature:
ImageFont.truetype(
    font=None,
    size=10,
    index=0,
    encoding='',
    layout_engine=None,
)
Docstring:
Load a TrueType or OpenType font from a file or file-like object,
and create a font object.
This function loads a font object from the given file or file-like
object, and creates a font object for a font of the given size.

Pillow uses FreeType to open font files. If you are opening many fonts
simultaneously on Windows, be aware that Windows limits the number of files
that can be open in C at once to 512. If you approach that limit, an
``OSError`` may be thrown, reporting that FreeType "cannot open resource".

This function requires the _imagingft service.

:param font: A filename or file-like object containing a TrueType font.
             If the file is not found in this filename, the loader may also
             search in other directories, such as the :file:`fonts/`
             directory on Windows or :file:`/Library/Fonts/`,
             :file:`/System/Lib